Initial Environment

In [2]:
%loadFromPOM pom.xml

In [3]:
import ca.uhn.fhir.context.FhirContext;
import ca.uhn.fhir.rest.client.api.IGenericClient;
import ca.uhn.fhir.rest.api.MethodOutcome;
import org.hl7.fhir.r4.model.*;

In [4]:
FhirContext ctx = FhirContext.forR4();
String serverBase = "http://localhost:8080/fhir";
IGenericClient client = ctx.newRestfulGenericClient(serverBase);

Search - By Type

In [6]:
// query using the generic client
Bundle response = client.search()
      .forResource(Patient.class)
      .where(Patient.BIRTHDATE.beforeOrEquals().day("1949-01-01"))
      .and(Patient.GENERAL_PRACTITIONER.hasChainedProperty(
            Organization.NAME.matches().value("東部醫療網路")))
      .returnBundle(Bundle.class)
      .execute();
// print the count of resources in the bundle
System.out.println(response.getTotal());

12


Forward Chaining

Resource type A (“WHAT”) has a query parameter of type ‘reference’ to resource type B

All query parameters for resource type B can be accessed via ‘chaining’

條件一：Resource A的search parameter包含一類型為reference(Resource B)

條件二：以類型B的任一Search parameter當作搜尋條件

舉例：Encounter的search parameter中，Patient為reference，即可使用Patient的search parameter "NAME"，尋找符合姓名搜尋條件之所有Encounter資料。(實務上，這是一個Master-Detail的觀念，一般假設搜尋結果為一對多；若搜尋結果為多對多，則後續必須針對搜尋結果作後處理。例如一個Patient有多個Encounter，一個Encounter有多個Condition)

In [5]:
// search patient Sanford861, Maile198
Bundle response = client.search()
      .forResource(Patient.class)
      .where(Patient.NAME.matches().value("Sanford861"))
      .and(Patient.NAME.matches().value("Maile198"))
      .returnBundle(Bundle.class)
      .execute();
// print patient information
for (Bundle.BundleEntryComponent entry : response.getEntry()) {
    Patient patient = (Patient) entry.getResource();
    System.out.println(patient.getId());
}

// search 2023-02-03 的當天，Sanford861, Maile198的Encounter資料 
// Encounter?date=2023-02-03&patient.name=Sanford861, Maile198
Bundle response = client.search()
      .forResource(Encounter.class)
      .where(Encounter.DATE.exactly().day("2023-02-03"))
      .and(Encounter.PATIENT.hasChainedProperty(Patient.NAME.matches().value("Sanford861")))
      .and(Encounter.PATIENT.hasChainedProperty(Patient.NAME.matches().value("Maile198")))
      .returnBundle(Bundle.class)
      .execute();

// print encounter information
Encounter encounter = null;
for (Bundle.BundleEntryComponent entry : response.getEntry()) {
    encounter = (Encounter) entry.getResource();
    System.out.println(encounter.getId());
}

// search 2023-02-03 的當天，Sanford861, Maile198的所有Condition資料
// Condition?encounter.date=2023-02-03&encounter.patient.name=Sanford861, Maile198

Reference encounterRef = new Reference("Encounter/" + encounter.getIdElement().getIdPart());
Bundle response = client.search()
      .forResource(Condition.class)
      .where(Condition.ENCOUNTER.hasChainedProperty(Encounter.DATE.exactly().day("2023-02-03")))
      .and(Condition.ENCOUNTER.hasChainedProperty(Encounter.PATIENT.hasChainedProperty(
            Patient.NAME.matches().value("Sanford861"))))
      .and(Condition.ENCOUNTER.hasChainedProperty(Encounter.PATIENT.hasChainedProperty(
            Patient.NAME.matches().value("Maile198"))))
      .returnBundle(Bundle.class)
      .execute();

// print condition information
for (Bundle.BundleEntryComponent entry : response.getEntry()) {
    Condition condition = (Condition) entry.getResource();
    System.out.println(condition.getId());
}

http://localhost:8080/fhir/Patient/34486/_history/2
http://localhost:8080/fhir/Encounter/35184/_history/1
http://localhost:8080/fhir/Condition/35185/_history/1
http://localhost:8080/fhir/Condition/35186/_history/1


Search - By plain URL(Reverse Chaining)

This can be useful if you have a URL you retrieved from somewhere else that you want to use as a search. This can also be useful if you need to make a search that isn't presently supported by one of the existing fluent methods (e.g. reverse chaining with _has).

Reverse Chaining為Forward Chaining的反向操作， FHIR API使用"_has"實作。使用情境為指定Detail狀態，取得符合該狀態的Master清單。例如想知道符合Condition code為314529007(SNOMED-CT 代碼系統中的 “藥物審查到期” - Medication review due)的病患清單，即可使用reverse chaining. 由於HAPI FHIR Java SDK並未實作"_has"，因此使用Plain URL為替代方案。

In [6]:
//GET [base]/Patient?_has:Condition:patient:code:314529007
//取得Condition的code為314529007的Patient資料
//目前沒有實作_has參數，所以用以下方式取得 

Bundle response = client.search()
      .byUrl("Patient?_has:Condition:patient:code=314529007")
      .returnBundle(Bundle.class)
      .execute();

// print patient information
for (Bundle.BundleEntryComponent entry : response.getEntry()) {
    Patient patient = (Patient) entry.getResource();
    System.out.println(patient.getId());
}


http://localhost:8080/fhir/Patient/24313/_history/2
http://localhost:8080/fhir/Patient/25006/_history/2
http://localhost:8080/fhir/Patient/25302/_history/2
http://localhost:8080/fhir/Patient/25553/_history/2
http://localhost:8080/fhir/Patient/25841/_history/2
http://localhost:8080/fhir/Patient/33641/_history/2
http://localhost:8080/fhir/Patient/34486/_history/2
http://localhost:8080/fhir/Patient/35251/_history/2
http://localhost:8080/fhir/Patient/35280/_history/2
http://localhost:8080/fhir/Patient/35577/_history/2
http://localhost:8080/fhir/Patient/37136/_history/2
http://localhost:8080/fhir/Patient/26404/_history/2
http://localhost:8080/fhir/Patient/37829/_history/2


(Reverse) Includes

Include referenced resources in the response bundle

選擇特定的Master,並將一或多個Detail資料一併回傳，最好為一對多；若狀況為多對多，則需要後續處理。

In [9]:
//GET [base]/ Patient?name=Sanford861&name=Maile198&_revinclude=Encounter:patient&_revinclude=Condition:patient
//取得Patient的name為Sanford861的資料，並且取得其Encounter,Condition資料

Bundle response = client.search()   
        .forResource(Patient.class) 
        .where(Patient.NAME.matches().value("Sanford861"))
        .and(Patient.NAME.matches().value("Maile198"))
        .revInclude(Encounter.INCLUDE_PATIENT)
        .revInclude(Condition.INCLUDE_PATIENT)
        .returnBundle(Bundle.class)
        .execute();

// print resource id in the bundle
for (Bundle.BundleEntryComponent entry : response.getEntry()) {
    Resource resource = entry.getResource();
    System.out.println(resource.getId());
}


http://localhost:8080/fhir/Patient/34486/_history/2
http://localhost:8080/fhir/Encounter/34574/_history/1
http://localhost:8080/fhir/Encounter/34830/_history/1
http://localhost:8080/fhir/Condition/34831/_history/1
http://localhost:8080/fhir/Condition/34832/_history/1
http://localhost:8080/fhir/Encounter/35090/_history/1
http://localhost:8080/fhir/Encounter/34965/_history/1
http://localhost:8080/fhir/Encounter/34582/_history/1
http://localhost:8080/fhir/Encounter/35223/_history/1
http://localhost:8080/fhir/Encounter/34971/_history/1
http://localhost:8080/fhir/Condition/34972/_history/1
http://localhost:8080/fhir/Encounter/34590/_history/1
http://localhost:8080/fhir/Encounter/34597/_history/1
http://localhost:8080/fhir/Condition/34598/_history/1
http://localhost:8080/fhir/Condition/34599/_history/1
http://localhost:8080/fhir/Condition/34600/_history/1
http://localhost:8080/fhir/Condition/34601/_history/1
http://localhost:8080/fhir/Encounter/35118/_history/1
http://localhost:8080/fhir/Con

Include

選擇特定的Detail,並將一或多個Master資料一併回傳，通常為多對多，則需要後續處理。

In [10]:
// GET [base]/Condition?code=314529007&_include =Condition:patient
// 取得Condition的code為314529007的Condition資料，並且包含patient資料

Bundle response = client.search()
      .forResource(Condition.class)
      .where(Condition.CODE.exactly().code("314529007"))
      .include(Condition.INCLUDE_PATIENT.asRecursive())
      .returnBundle(Bundle.class)
      .execute();
      
// print resource id in the bundle
for (Bundle.BundleEntryComponent entry : response.getEntry()) {
    Resource resource = entry.getResource();
    System.out.println(resource.getId());
}


http://localhost:8080/fhir/Condition/24674/_history/1
http://localhost:8080/fhir/Condition/24559/_history/1
http://localhost:8080/fhir/Condition/24384/_history/1
http://localhost:8080/fhir/Condition/24340/_history/1
http://localhost:8080/fhir/Condition/24404/_history/1
http://localhost:8080/fhir/Condition/24758/_history/1
http://localhost:8080/fhir/Condition/24860/_history/1
http://localhost:8080/fhir/Condition/24906/_history/1
http://localhost:8080/fhir/Condition/25127/_history/1
http://localhost:8080/fhir/Condition/25236/_history/1
http://localhost:8080/fhir/Condition/25089/_history/1
http://localhost:8080/fhir/Condition/25037/_history/1
http://localhost:8080/fhir/Condition/25180/_history/1
http://localhost:8080/fhir/Condition/25444/_history/1
http://localhost:8080/fhir/Condition/25345/_history/1
http://localhost:8080/fhir/Condition/25524/_history/1
http://localhost:8080/fhir/Condition/25670/_history/1
http://localhost:8080/fhir/Condition/25559/_history/1
http://localhost:8080/fhir/C

Search - Multi-valued Parameters (ANY/OR)

In [9]:
Patient patient = null;

// This leads to a URL resembling http://base/Patient?family=Mildred587,Farrell962
response = client.search()
      .forResource(Patient.class)
      .where(Patient.FAMILY.matches().values("Mildred587", "Farrell962"))
      .returnBundle(Bundle.class)
      .execute();
// print the results
for (Bundle.BundleEntryComponent entry : response.getEntry()) {
    patient = (Patient) entry.getResource();
}

System.out.println(patient.getGeneralPractitioner().get(0).getReference());

Organization/38113
